In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spooky-author-identification/train.zip
/kaggle/input/spooky-author-identification/test.zip
/kaggle/input/spooky-author-identification/sample_submission.zip
/kaggle/input/capstone-karthik/new_train.csv


In [10]:
!pip install -q --upgrade pip

## installing the latest transformers version from pip
!pip install --use-feature=2020-resolver -q transformers==3.0.2
import transformers

## installing Google Translator package
!pip install -q googletrans

In [11]:
import gc
import os
import random
import transformers
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K

from googletrans import Translator
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModel

print(f"TensorFlow version: {tf.__version__}")
print(f"Transformers version: {transformers.__version__}")

warnings.filterwarnings("ignore")

TensorFlow version: 2.2.0
Transformers version: 3.0.2


In [12]:
class Configuration():
    """
    All configuration for running an experiment
    """
    def __init__(
        self,
        model_name,
        truncation = True,
        translation = True,
        max_length = 64,
        padding = True,
        batch_size = 128,
        epochs = 5,
        learning_rate = 1e-5,
        metrics = ["sparse_categorical_accuracy"],
        verbose = 1,
        train_splits = 5,
        accelerator = "TPU",
        myluckynumber = 13
    ):
        # seed and accelerator
        self.SEED = myluckynumber
        self.ACCELERATOR = accelerator

        # paths
        self.PATH_TRAIN = Path("../input/capstone-karthik/new_train.csv")
        self.PATH_TEST  = Path("../input/spooky-author-identification/test.zip")

        # splits
        self.TRAIN_SPLITS = train_splits

        
        # model configuration
        self.MODEL_NAME = model_name
        # self.TRANSLATION = translation
        self.TOKENIZER = AutoTokenizer.from_pretrained(self.MODEL_NAME)

        # model hyperparameters
        self.MAX_LENGTH = max_length
        self.PAD_TO_MAX_LENGTH = padding
        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.LEARNING_RATE = learning_rate
        self.METRICS = metrics
        self.VERBOSE = verbose
        
        # initializing accelerator
        self.initialize_accelerator()

    def initialize_accelerator(self):
        """
        Initializing accelerator
        """
        # checking TPU first
        if self.ACCELERATOR == "TPU":
            print("Connecting to TPU")
            try:
                tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
                print(f"Running on TPU {tpu.master()}")
            except ValueError:
                print("Could not connect to TPU")
                tpu = None

            if tpu:
                try:
                    print("Initializing TPU")
                    tf.config.experimental_connect_to_cluster(tpu)
                    tf.tpu.experimental.initialize_tpu_system(tpu)
                    self.strategy = tf.distribute.experimental.TPUStrategy(tpu)
                    self.tpu = tpu
                    print("TPU initialized")
                except _:
                    print("Failed to initialize TPU")
            else:
                print("Unable to initialize TPU")
                self.ACCELERATOR = "GPU"

        # default for CPU and GPU
        if self.ACCELERATOR != "TPU":
            print("Using default strategy for CPU and single GPU")
            self.strategy = tf.distribute.get_strategy()

        # checking GPUs
        if self.ACCELERATOR == "GPU":
            print(f"GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")

        # defining replicas
        self.AUTO = tf.data.experimental.AUTOTUNE
        self.REPLICAS = self.strategy.num_replicas_in_sync
        print(f"REPLICAS: {self.REPLICAS}")

In [13]:
def encode_text(df, tokenizer, max_len, padding):
    """
    Preprocessing textual data into encoded tokens.
    """
    text = df["text"].values.tolist()

    # encoding text using tokenizer of the model
    text_encoded = tokenizer.batch_encode_plus(
        text,
        pad_to_max_length = padding,
        max_length = max_len
    )

    return text_encoded


def get_tf_dataset(X, y, auto, labelled = True, repeat = False, shuffle = False, batch_size = 128):
    """
    Creating tf.data.Dataset for TPU.
    """
    if labelled:
        ds = (tf.data.Dataset.from_tensor_slices((X["input_ids"], y)))
    else:
        ds = (tf.data.Dataset.from_tensor_slices(X["input_ids"]))

    if repeat:
        ds = ds.repeat()

    if shuffle:
        ds = ds.shuffle(2048)

    ds = ds.batch(batch_size)
    ds = ds.prefetch(auto)

    return ds

In [14]:
def build_model(model_name, max_len, learning_rate, metrics):
    """
    Building the Deep Learning architecture
    """
    # defining encoded inputs
    input_ids = Input(shape = (max_len,), dtype = tf.int32, name = "input_ids")
    
    # defining transformer model embeddings
    transformer_model = TFAutoModel.from_pretrained(model_name)
    transformer_embeddings = transformer_model(input_ids)[0]

    # defining output layer
    output_values = Dense(3, activation = "softmax")(transformer_embeddings[:, 0, :])

    # defining model
    model = Model(inputs = input_ids, outputs = output_values)
    opt = Adam(learning_rate = learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    metrics = metrics

    model.compile(optimizer = opt, loss = loss, metrics = metrics)

    return model

In [15]:
def run_model(config):
    """
    Running the model
    """
    ## reading data
    df_train = pd.read_csv(config.PATH_TRAIN)
    df_test = pd.read_csv(config.PATH_TEST)

     # stratified K-fold on language and label
    skf = StratifiedKFold(n_splits = config.TRAIN_SPLITS, shuffle = True, random_state = config.SEED)

    # initializing predictions
    preds_oof = np.zeros((df_train.shape[0], 3))
    preds_test = np.zeros((df_test.shape[0], 3))
    acc_oof = []

    # iterating over folds
    for (fold, (train_index, valid_index)) in enumerate(skf.split(df_train, df_train.author)):
        # initializing TPU
        if config.ACCELERATOR == "TPU":
            if config.tpu:
                config.initialize_accelerator()

        # building model
        K.clear_session()
        with config.strategy.scope():
            model = build_model(config.MODEL_NAME, config.MAX_LENGTH, config.LEARNING_RATE, config.METRICS)
            if fold == 0:
                print(model.summary())

        print("\n")
        print("#" * 19)
        print(f"##### Fold: {fold + 1} #####")
        print("#" * 19)

        # splitting data into training and validation
        X_train = df_train.iloc[train_index]
        X_valid = df_train.iloc[valid_index]

        y_train = X_train.author.values
        y_valid = X_valid.author.values

        print("\nTokenizing")

        # encoding text data using tokenizer
        X_train_encoded = encode_text(df = X_train, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)
        X_valid_encoded = encode_text(df = X_valid, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)

        # creating TF Dataset
        ds_train = get_tf_dataset(X_train_encoded, y_train, config.AUTO, repeat = True, shuffle = True, batch_size = config.BATCH_SIZE * config.REPLICAS)
        ds_valid = get_tf_dataset(X_valid_encoded, y_valid, config.AUTO, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)

        n_train = X_train.shape[0]

        if fold == 0:
            X_test_encoded = encode_text(df = df_test, tokenizer = config.TOKENIZER, max_len = config.MAX_LENGTH, padding = config.PAD_TO_MAX_LENGTH)

        # saving model at best accuracy epoch
        sv = tf.keras.callbacks.ModelCheckpoint(
            "model.h5",
            monitor = "val_sparse_categorical_accuracy",
            verbose = 0,
            save_best_only = True,
            save_weights_only = True,
            mode = "max",
            save_freq = "epoch"
        )

        print("\nTraining")

        # training model
        model_history = model.fit(
            ds_train,
            epochs = config.EPOCHS,
            callbacks = [sv],
            steps_per_epoch = n_train / config.BATCH_SIZE // config.REPLICAS,
            validation_data = ds_valid,
            verbose = config.VERBOSE
        )

        print("\nValidating")

        # scoring validation data
        model.load_weights("model.h5")
        ds_valid = get_tf_dataset(X_valid_encoded, -1, config.AUTO, labelled = False, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)

        preds_valid = model.predict(ds_valid, verbose = config.VERBOSE)
        acc = accuracy_score(y_valid, np.argmax(preds_valid, axis = 1))

        preds_oof[valid_index] = preds_valid
        acc_oof.append(acc)

        print("\nInferencing")

        # scoring test data
        ds_test = get_tf_dataset(X_test_encoded, -1, config.AUTO, labelled = False, batch_size = config.BATCH_SIZE * config.REPLICAS * 4)
        preds_test += model.predict(ds_test, verbose = config.VERBOSE) / config.TRAIN_SPLITS

        print(f"\nFold {fold + 1} Accuracy: {round(acc, 4)}\n")

        g = gc.collect()

    # overall CV score and standard deviation
    print(f"\nCV Mean Accuracy: {round(np.mean(acc_oof), 4)}")
    print(f"CV StdDev Accuracy: {round(np.std(acc_oof), 4)}\n")

    return preds_oof, preds_test

In [16]:
# Final Model: XLM Roberta Large
# config_1 = Configuration("jplu/tf-xlm-roberta-large", translation = False, max_length = 84, batch_size = 64, epochs = 16, train_splits = 4)
# preds_train_1, preds_test_1 = run_model(config_1)

Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8
Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 84)]              0         
_________________________________________________________________
tf_roberta_model (TFRobertaM ((None, 84, 1024), (None, 559890432 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 559,893,507
Trainable params: 559,893,507
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing

Training
Epoch 1/16
28/28 [==============================] - 106s 4s/step - sparse_categorical_accuracy: 0.4190 - loss: 1.0730 - val_sparse_categorical_accuracy: 0.

28/28 [==============================] - 31s 1s/step - sparse_categorical_accuracy: 0.8276 - loss: 0.7201 - val_sparse_categorical_accuracy: 0.8022 - val_loss: 0.7443
Epoch 6/16
28/28 [==============================] - 31s 1s/step - sparse_categorical_accuracy: 0.8521 - loss: 0.6973 - val_sparse_categorical_accuracy: 0.8278 - val_loss: 0.7188
Epoch 7/16
28/28 [==============================] - 21s 747ms/step - sparse_categorical_accuracy: 0.8665 - loss: 0.6830 - val_sparse_categorical_accuracy: 0.8174 - val_loss: 0.7295
Epoch 8/16
28/28 [==============================] - 31s 1s/step - sparse_categorical_accuracy: 0.8815 - loss: 0.6674 - val_sparse_categorical_accuracy: 0.8321 - val_loss: 0.7153
Epoch 9/16
28/28 [==============================] - 21s 747ms/step - sparse_categorical_accuracy: 0.8966 - loss: 0.6541 - val_sparse_categorical_accuracy: 0.8264 - val_loss: 0.7203
Epoch 10/16
28/28 [==============================] - 31s 1s/step - sparse_categorical_accuracy: 0.9072 - loss: 0.64

In [17]:
# Model: Bert Base Cased
# config_2 = Configuration("bert-base-cased", max_length = 84, batch_size = 64, epochs = 16, train_splits = 4)
# preds_train_2, preds_test_2 = run_model(config_2)


Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8
Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 84)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 84, 768), (None,  108310272 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 108,312,579
Trainable params: 108,312,579
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing

Training
Epoch 1/16
28/28 [==============================] - 57s 2s/step - sparse_categorical_accuracy: 0.4107 - loss: 1.0810 - val_sparse_categorical_accuracy: 0.5

28/28 [==============================] - 10s 346ms/step - sparse_categorical_accuracy: 0.8514 - loss: 0.7019 - val_sparse_categorical_accuracy: 0.8037 - val_loss: 0.7394
Epoch 6/16
28/28 [==============================] - 10s 345ms/step - sparse_categorical_accuracy: 0.8654 - loss: 0.6855 - val_sparse_categorical_accuracy: 0.8184 - val_loss: 0.7264
Epoch 7/16
28/28 [==============================] - 10s 344ms/step - sparse_categorical_accuracy: 0.8906 - loss: 0.6642 - val_sparse_categorical_accuracy: 0.8217 - val_loss: 0.7228
Epoch 8/16
28/28 [==============================] - 7s 266ms/step - sparse_categorical_accuracy: 0.8982 - loss: 0.6548 - val_sparse_categorical_accuracy: 0.8192 - val_loss: 0.7244
Epoch 9/16
28/28 [==============================] - 9s 338ms/step - sparse_categorical_accuracy: 0.9139 - loss: 0.6394 - val_sparse_categorical_accuracy: 0.8290 - val_loss: 0.7156
Epoch 10/16
28/28 [==============================] - 10s 351ms/step - sparse_categorical_accuracy: 0.9211 - 

In [18]:
# Model: Bert Base Uncased
# config_3 = Configuration("bert-base-uncased", max_length = 84, batch_size = 64, epochs = 16, train_splits = 4)
# preds_train_3, preds_test_3 = run_model(config_3)


Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8
Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 84)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 84, 768), (None,  109482240 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 768)]             0         
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 109,484,547
Trainable params: 109,484,547
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing

Training
Epoch 1/16
28/28 [==============================] - 57s 2s/step - sparse_categorical_accuracy: 0.4877 - loss: 1.0388 - val_sparse_categorical_accuracy: 0.6

28/28 [==============================] - 10s 356ms/step - sparse_categorical_accuracy: 0.8382 - loss: 0.7129 - val_sparse_categorical_accuracy: 0.8074 - val_loss: 0.7389
Epoch 6/16
28/28 [==============================] - 10s 360ms/step - sparse_categorical_accuracy: 0.8547 - loss: 0.6960 - val_sparse_categorical_accuracy: 0.8155 - val_loss: 0.7316
Epoch 7/16
28/28 [==============================] - 10s 362ms/step - sparse_categorical_accuracy: 0.8782 - loss: 0.6740 - val_sparse_categorical_accuracy: 0.8157 - val_loss: 0.7275
Epoch 8/16
28/28 [==============================] - 11s 375ms/step - sparse_categorical_accuracy: 0.8910 - loss: 0.6607 - val_sparse_categorical_accuracy: 0.8306 - val_loss: 0.7177
Epoch 9/16
28/28 [==============================] - 10s 356ms/step - sparse_categorical_accuracy: 0.9039 - loss: 0.6491 - val_sparse_categorical_accuracy: 0.8337 - val_loss: 0.7139
Epoch 10/16
28/28 [==============================] - 10s 347ms/step - sparse_categorical_accuracy: 0.9148 

In [19]:
# Model: Bert Large Cased
# config_4 = Configuration("bert-large-cased", max_length = 84, batch_size = 64, epochs = 16, train_splits = 4)
# preds_train_4, preds_test_4 = run_model(config_4)


Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8
Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 84)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 84, 1024), (None, 333579264 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 333,582,339
Trainable params: 333,582,339
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing

Training
Epoch 1/16
28/28 [==============================] - 106s 4s/step - sparse_categorical_accuracy: 0.4948 - loss: 1.0305 - val_sparse_categorical_accuracy: 0.

28/28 [==============================] - 19s 681ms/step - sparse_categorical_accuracy: 0.9024 - loss: 0.6520 - val_sparse_categorical_accuracy: 0.8294 - val_loss: 0.7143
Epoch 6/16
28/28 [==============================] - 27s 972ms/step - sparse_categorical_accuracy: 0.9182 - loss: 0.6345 - val_sparse_categorical_accuracy: 0.8505 - val_loss: 0.6970
Epoch 7/16
28/28 [==============================] - 19s 681ms/step - sparse_categorical_accuracy: 0.9300 - loss: 0.6232 - val_sparse_categorical_accuracy: 0.8368 - val_loss: 0.7109
Epoch 8/16
28/28 [==============================] - 27s 968ms/step - sparse_categorical_accuracy: 0.9420 - loss: 0.6115 - val_sparse_categorical_accuracy: 0.8529 - val_loss: 0.6924
Epoch 9/16
28/28 [==============================] - 19s 683ms/step - sparse_categorical_accuracy: 0.9456 - loss: 0.6066 - val_sparse_categorical_accuracy: 0.8496 - val_loss: 0.6964
Epoch 10/16
28/28 [==============================] - 19s 679ms/step - sparse_categorical_accuracy: 0.9555 

In [20]:
# Model: Bert Large Uncased
config_5 = Configuration("bert-large-uncased", max_length = 84, batch_size = 64, epochs = 16, train_splits = 4)
preds_train_5, preds_test_5 = run_model(config_5)


Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8
Connecting to TPU
Running on TPU grpc://10.0.0.2:8470
Initializing TPU
TPU initialized
REPLICAS: 8



Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 84)]              0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 84, 1024), (None, 335141888 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 335,144,963
Trainable params: 335,144,963
Non-trainable params: 0
_________________________________________________________________
None


###################
##### Fold: 1 #####
###################

Tokenizing

Training
Epoch 1/16
28/28 [==============================] - 106s 4s/step - sparse_categorical_accuracy: 0.5882 - loss: 0.9522 - val_sparse_categorical_accuracy: 0.

28/28 [==============================] - 19s 679ms/step - sparse_categorical_accuracy: 0.8477 - loss: 0.7030 - val_sparse_categorical_accuracy: 0.7933 - val_loss: 0.7540
Epoch 6/16
28/28 [==============================] - 28s 985ms/step - sparse_categorical_accuracy: 0.8634 - loss: 0.6876 - val_sparse_categorical_accuracy: 0.8006 - val_loss: 0.7443
Epoch 7/16
28/28 [==============================] - 27s 973ms/step - sparse_categorical_accuracy: 0.8866 - loss: 0.6645 - val_sparse_categorical_accuracy: 0.8210 - val_loss: 0.7246
Epoch 8/16
28/28 [==============================] - 27s 970ms/step - sparse_categorical_accuracy: 0.9037 - loss: 0.6494 - val_sparse_categorical_accuracy: 0.8360 - val_loss: 0.7114
Epoch 9/16
28/28 [==============================] - 28s 988ms/step - sparse_categorical_accuracy: 0.9157 - loss: 0.6379 - val_sparse_categorical_accuracy: 0.8368 - val_loss: 0.7092
Epoch 10/16
28/28 [==============================] - 28s 991ms/step - sparse_categorical_accuracy: 0.9246 